# VBA Files

### Pulling Excel Files from the Website

In [1]:
# There are weekly files from 2014 to present on https://www.benefits.va.gov/reports/detailed_claims_data.asp

# There are historical files at https://www.benefits.va.gov/REPORTS/mmwr/historical/{year}/index.asp from 2004 to 2013

# Presumably the first step will be just pulling all of them

### Libraries/Setup

In [1]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import win32com.client
os.chdir(r'C:\Users\admin\Documents\Python Scripts\veteranLawData')

### Get File URLs From By-Year Pages

In [2]:
def pullFilesByYear(year):
    url= f'https://www.benefits.va.gov/REPORTS/mmwr/historical/{str(year)}/index.asp'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    filesToGet=[]
    for link in soup.findAll('a'):
        try:
            if "xls" in (link.get('href')):
                filesToGet.append(link.get('href'))
        except:
            pass
    return(filesToGet)

yearRange=list(range(2004, 2014))
#files=pullTheFiles(2004)
filesList=list(map(pullFilesByYear, yearRange))
flatFileList= [item for sublist in filesList for item in sublist]

### Get File URLs From Top Page

In [3]:
def pullFilesFromSite():
    url='https://www.benefits.va.gov/reports/detailed_claims_data.asp'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    filesToGet=[]
    for link in soup.findAll('a'):
        try:
            if "xls" in (link.get('href')):
                filesToGet.append(link.get('href'))
        except:
            pass
    return(filesToGet)

filesList2=pullFilesFromSite()
    

### Check We Have The Right Number From By-Year Archive Pages: 2008 Is Legitimately Missing Three, I Checked

In [4]:
for fileList in filesList:
    print(len(fileList))

52
52
52
52
49
52
52
52
52
53


### Check We Have The Right Number from Top, Recent Page

In [5]:
files2018=[i.split("_")[1] for i in filesList2 if ("2018" in i and "18" in i.split("_")[1])]
len(files2018)

54

### This Looks Pretty Good

In [6]:
files2018.sort()
files2018

['01-01-18.xlsx',
 '01-08-18.xlsx',
 '01-16-18.xlsx',
 '01-22-18.xlsx',
 '01-29-18.xlsx',
 '02-05-18.xlsx',
 '02-12-18.xlsx',
 '02-19-18.xlsx',
 '02-26-18.xlsx',
 '03-05-18.xlsx',
 '03-12-18.xlsx',
 '03-19-18.xlsx',
 '03-26-18.xlsx',
 '04-02-18.xlsx',
 '04-09-18.xlsx',
 '04-16-18.xlsx',
 '04-23-18.xlsx',
 '04-28-18.xlsx',
 '05-07-18.xlsx',
 '05-14-18.xlsx',
 '05-21-18.xlsx',
 '05-29-18.xlsx',
 '06-04-18.xlsx',
 '06-11-18.xlsx',
 '06-18-18.xlsx',
 '06-25-18.xlsx',
 '07-09-18.xlsx',
 '07-14-18.xlsx',
 '07-23-18.xlsx',
 '07-30-18.xlsx',
 '08-06-18.xlsx',
 '08-13-18.xlsx',
 '08-20-2018.xlsx',
 '08-27-2018.xlsx',
 '09-04-18.xlsx',
 '09-10-18.xlsx',
 '09-17-18.xlsx',
 '09-24-18.xlsx',
 '10-01-18.xlsx',
 '10-08-18',
 '10-15-18',
 '10-22-18',
 '10-29-18',
 '11-05-18',
 '11-13-18',
 '11-19-18',
 '11-26-18.xlsx',
 '12-03-18.xlsx',
 '12-10-18.xlsx',
 '12-17-18.xlsx',
 '12-18-17.xlsx',
 '12-24-18.xlsx',
 '12-31-18.xlsx',
 '7-2-18.xlsx']

### Append The Lists

In [7]:
allList= flatFileList + filesList2

### Pull The Files

In [5]:
#there are two different formats for how the file URLs are structured; this picks up both
def pullFile(url):
    try:
        cleanLink='https://www.benefits.va.gov'+url
        r = requests.get(cleanLink, allow_redirects=True)
        name=cleanLink.split("/")[-1]
        open(name, 'wb').write(r.content)
        return(name)
    except:
        try: 
            r = requests.get(url, allow_redirects=True)
            name=url.split("/")[-1]
            open(name, 'wb').write(r.content)
        except:
            print(url)

listOfFileNames=list(map(pullFile, allList))

### Read Them Into A Dictionary

In [ ]:
#there are a few different formats for the content we want, identifiable by tab names 
#and whether it errors out for being encrypted
def getEncyptedFiles(fileName):
    fullFileName=os.getcwd()+'\\'+fileName
    #fileName=r"C:\Users\admin\Documents\Python Scripts\veteranLawData\MMWL_01-04-14.xls"
    excel = win32com.client.Dispatch('Excel.Application')
    workbook = excel.Workbooks.open(fullFileName)
    try:
        sheet = workbook.WorkSheets('Traditional Aggregate')
    except:
        sheet = workbook.WorkSheets(r'TA-Regional Office')
    content = sheet.Range(sheet.Cells(9, 2), sheet.Cells(80, 18)).Value 
    # Transfer content to pandas dataframe
    df = pd.DataFrame(list(content))
    df.columns = df.iloc[0]
    workbook.Close(True)
    return(df)

def readBasicFile(fileName):
    try:
        xl=pd.ExcelFile(fileName)
        if len(xl.sheet_names)==1:
            df=pd.read_excel(fileName)
        if 'Final Aggregate' in xl.sheet_names:
            df=pd.read_excel(fileName, sheet_name='Final Aggregate', header=7)
        if 'Traditional Aggregate' in xl.sheet_names:
            df=pd.read_excel(fileName, sheet_name='Traditional Aggregate', header=7)
        #print("worked1")
        return(df)
    except:
        try:
            df=getEncyptedFiles(fileName)
            #print("worked2")
            return(df)
        except:
            print(f"failed on {fileName}")
        
downloadedFiles=[i for i os.listdir() if "xls" in i]
listOfDFs=list(map(readBasicFile, downloadedFiles))


In [ ]:
dictionaryOfFiles = dict(zip(filesNameList, listOfDFs))

### Append Into Master DF

In [81]:


masterDF=pd.DataFrame()
for key in dictionaryOfFiles.keys():
    try:
        df=dictionaryOfFiles[key]
        df['fileName']=key
        listOfPhrasesToFind=["As of", "For the Week"]
        foundPhrase=[i for i in df.columns if any(substring in i for substring in listOfPhrasesToFind)]
        dropPhrase=[i for i in df.columns if "Percent" in i]
        #print(len(foundPhrase))
        #df["forTheWeek"]=df[foundPhrase[0]]
        #df=df.drop([foundPhrase[0]], axis=1)
        #df=df.drop(dropPhrase, axis=1)
        masterDF=masterDF.append(df, sort=True)
        #print(key)
    except:
        print(f'failed on {key}')
        

### It looks like we're not obviously clipping off the bottoms of any of the tables; we'll need to look further at both extremes

In [82]:
(masterDF['fileName'].value_counts()).value_counts()

Series([], Name: fileName, dtype: int64)

### Right now: it looks like 04-09 is working; after that we need to look at the format of the 09 worksheets to modify it; there's also some column cleaning we may need to do for the 04-09 group; this is a good stopping point

### Are we missing lines? spot checked first and last, looked good, getting weird value counts for the regional variable, but the overall counts don't have too much variation

In [23]:
seriesOfLength=pd.Series()

for file in masterDF['fileName'].unique():
    s= pd.Series([len(masterDF.loc[masterDF['fileName']==file])], index=[file])
    seriesOfLength=seriesOfLength.append(s)

In [24]:
seriesOfLength.value_counts()

71    84
72    83
70    78
68    24
78    15
73    11
dtype: int64

In [25]:
seriesOfLength.loc[seriesOfLength==68]

010504.xls    68
040504.xls    68
012004.xls    68
041204.xls    68
041904.xls    68
012604.xls    68
042604.xls    68
020204.xls    68
050304.xls    68
020904.xls    68
051004.xls    68
021704.xls    68
051704.xls    68
022304.xls    68
052404.xls    68
030104.xls    68
060104.xls    68
030804.xls    68
060704.xls    68
031504.xls    68
061404.xls    68
032204.xls    68
062104.xls    68
032904.xls    68
dtype: int64

### Looking at One With Count 68 : Looks Good

In [27]:
masterDF.loc[masterDF['fileName']=='021704.xls']

,,Rating Cases Pending,Rating Claims Pending,* Scorecard Non-Rating Cases Pending,* Scorecard Non-Rating Claims Pending,C&P Claims over 180 Days,C&P Claims over 180 Days in WIPP,C&P Work Items over 180 Days,COE,COEs Issued,...,Total Appeals Requiring Adjudicative Action,Total Appeals Requiring Adjudicative action,Total C&P Pending,Total C&P Pending in Wipp,Total C&P Work Items Pending,Unnamed: 18,VACOLS Appeals,VACOLS Appeals includes AMC & Travel Board,fileName,forTheWeek
0,NaT,NaN,NaN,NaN,NaN,NaN,102909.000000,NaN,NaT,1906-11-01 00:00:00,...,NaN,124637.000000,NaN,478130.000000,NaN,NaN,NaN,145668.000000,021704.xls,USA
1,NaT,NaN,NaN,NaN,NaN,NaN,104386.000000,NaN,NaT,1906-11-01 00:00:00,...,NaN,125006.000000,NaN,470643.000000,NaN,NaN,NaN,145132.000000,021704.xls,Last Week
2,NaT,NaN,NaN,NaN,NaN,NaN,-1477.000000,NaN,NaT,00:00:00,...,NaN,-369.000000,NaN,7487.000000,NaN,NaN,NaN,536.000000,021704.xls,Change from Last Week
3,NaT,NaN,NaN,NaN,NaN,NaN,-0.014149,NaN,NaT,NaN,...,NaN,-0.002952,NaN,0.015908,NaN,NaN,NaN,0.003693,021704.xls,Percent Change
4,NaT,NaN,NaN,NaN,NaN,NaN,118461.000000,NaN,NaT,1906-11-01 00:00:00,...,NaN,97678.000000,NaN,472859.000000,NaN,NaN,NaN,121596.000000,021704.xls,Last Year
5,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,021704.xls,EASTERN AREA
6,NaT,NaN,NaN,NaN,NaN,NaN,1890.000000,NaN,NaT,NaN,...,NaN,1165.000000,NaN,6161.000000,NaN,NaN,NaN,1466.000000,021704.xls,Baltimore Regional Office
7,NaT,NaN,NaN,NaN,NaN,NaN,2757.000000,NaN,NaT,NaN,...,NaN,1568.000000,NaN,7641.000000,NaN,NaN,NaN,1806.000000,021704.xls,Boston Regional Office
8,NaT,NaN,NaN,NaN,NaN,NaN,2147.000000,NaN,NaT,NaN,...,NaN,2040.000000,NaN,6959.000000,NaN,NaN,NaN,2319.000000,021704.xls,Buffalo Regional Office
9,NaT,NaN,NaN,NaN,NaN,NaN,3610.000000,NaN,NaT,NaN,...,NaN,3110.000000,NaN,12994.000000,NaN,NaN,NaN,3960.000000,021704.xls,Cleveland Regional Office


### We're getting "XLRDError: Workbook is encrypted" for many later workbooks, we need a different package

In [75]:
dataframe

,None,Avg. Days Pending,# Pending,# Pending over 125 days,Percent Pending > 125 days,# Pending,# Pending over 125 days,Percent Pending > 125 days,# Pending,# Pending over 125 days,Percent Pending > 125 days,# Pending,# Pending over 125 days,Percent Pending > 125 days,# Pending,# Pending,# Pending
0,None,Avg. Days Pending,# Pending,# Pending over 125 days,Percent Pending > 125 days,# Pending,# Pending over 125 days,Percent Pending > 125 days,# Pending,# Pending over 125 days,Percent Pending > 125 days,# Pending,# Pending over 125 days,Percent Pending > 125 days,# Pending,# Pending,# Pending
1,USAC,323.4,636274,388178,0.61008,406934,289271,0.710855,74142,59177,0.798158,171948,90400,0.52574,30507,1971,264066
2,Eastern Area,330.3,125918,78337,0.622127,68475,48656,0.710566,14863,12479,0.839602,71077,20879,0.293752,25109,384,46396
3,Baltimore,493.9,9518,7042,0.739861,6587,5782,0.87779,1166,1050,0.900515,51016,6641,0.130175,1,3,3459
4,Boston,343.5,6585,4038,0.613212,6162,4541,0.736936,1868,1476,0.79015,1164,904,0.776632,1,2,4377
5,Buffalo,227.2,7883,4832,0.612965,4464,2951,0.661066,357,314,0.879552,497,295,0.593561,2,5,1571
6,Cleveland,341.6,19187,12776,0.665868,11352,8923,0.786029,2114,2056,0.972564,3637,2290,0.62964,5,164,10062
7,Detroit,136,14725,9038,0.613786,5096,2289,0.449176,1133,948,0.836717,2779,2021,0.72724,3,75,5989
8,Hartford,160.9,3480,2047,0.588218,2039,1083,0.531143,314,197,0.627389,1259,1143,0.907863,1,0,860
9,Indianapolis,394.3,11051,6750,0.610804,7241,5762,0.795746,1228,934,0.760586,1498,955,0.637517,2,123,5405


### This is working fine, the question is, can we actually hardcode 7 or do we need to make this more flexible?

In [38]:
dictionaryOfFiles['NewMMWL_100509.xls']

,Claims Pending,Pending over 125 days,Percent Pending over 125 days,Pending,Pending over 125 days.1,Percent Pending over 125 days.1,Pending,Pending over 125 days.2,Percent Pending over 125 days.2,Pending.1,Pending over 125 days.3,Percent Pending over 125 days.3,Claims Pending.1,Claims Pending.2,Pending.1,fileName
USA,388774,142652,0.366929,148042,47131,0.318362,22129,9108,0.411587,27635,15176,0.549159,3799,457,174891,NewMMWL_100509.xls
EASTERN AREA,82276,33396,0.405902,27441,7754,0.28257,5627,2891,0.513773,5015,2993,0.59681,771,106,31196,NewMMWL_100509.xls
Baltimore,8567,4320,0.504261,3202,1917,0.598688,1933,1232,0.637351,459,297,0.647059,28,15,2106,NewMMWL_100509.xls
Boston,4253,1213,0.28521,889,178,0.200225,378,125,0.330688,305,204,0.668852,71,5,2142,NewMMWL_100509.xls
Buffalo,5096,2290,0.449372,1751,349,0.199315,278,101,0.363309,249,92,0.369478,151,4,776,NewMMWL_100509.xls
Cleveland,10121,4018,0.396996,2603,290,0.11141,519,81,0.156069,480,290,0.604167,7,0,6107,NewMMWL_100509.xls
Detroit,11577,5650,0.488037,2891,581,0.200969,418,200,0.478469,951,731,0.768665,19,1,3884,NewMMWL_100509.xls
Hartford,1525,384,0.251803,979,62,0.0633299,117,65,0.555556,72,33,0.458333,7,5,1068,NewMMWL_100509.xls
Indianapolis,10309,4752,0.460956,4232,1719,0.406191,524,348,0.664122,604,227,0.375828,41,15,2842,NewMMWL_100509.xls
Manchester,1272,417,0.32783,426,51,0.119718,2,0,0,104,36,0.346154,59,0,667,NewMMWL_100509.xls


In [78]:
os.getcwd()

'C:\\Users\\admin\\Documents\\Python Scripts\\veteranLawData'